In [83]:
#importing library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import r2_score
import optuna
import re

In [84]:
#loading the dataset
df=pd.read_csv('/kaggle/input/SolarEnergy/SolarPrediction.csv')
#loading the dataset
df

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 12:00:00 AM,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00
1,1475229023,9/29/2016 12:00:00 AM,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00
2,1475228726,9/29/2016 12:00:00 AM,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00
3,1475228421,9/29/2016 12:00:00 AM,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00
4,1475228124,9/29/2016 12:00:00 AM,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00
...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,12/1/2016 12:00:00 AM,00:20:04,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00
32682,1480587301,12/1/2016 12:00:00 AM,00:15:01,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00
32683,1480587001,12/1/2016 12:00:00 AM,00:10:01,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00
32684,1480586702,12/1/2016 12:00:00 AM,00:05:02,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32686 entries, 0 to 32685
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   UNIXTime                32686 non-null  int64  
 1   Data                    32686 non-null  object 
 2   Time                    32686 non-null  object 
 3   Radiation               32686 non-null  float64
 4   Temperature             32686 non-null  int64  
 5   Pressure                32686 non-null  float64
 6   Humidity                32686 non-null  int64  
 7   WindDirection(Degrees)  32686 non-null  float64
 8   Speed                   32686 non-null  float64
 9   TimeSunRise             32686 non-null  object 
 10  TimeSunSet              32686 non-null  object 
dtypes: float64(4), int64(3), object(4)
memory usage: 2.7+ MB


# Checking for Missing Values

In [86]:
df.isna().sum()

UNIXTime                  0
Data                      0
Time                      0
Radiation                 0
Temperature               0
Pressure                  0
Humidity                  0
WindDirection(Degrees)    0
Speed                     0
TimeSunRise               0
TimeSunSet                0
dtype: int64

# Feature Engineering

In [87]:
df['Day']=df['Data'].apply(lambda x:x.split('/')[0])
df['Month']=df['Data'].apply(lambda x:x.split('/')[1])


In [88]:
df['Year']=df['Data'].apply(lambda x:x.split('/')[2].split(' ')[0])

In [89]:
df['Time']

0        23:55:26
1        23:50:23
2        23:45:26
3        23:40:21
4        23:35:24
           ...   
32681    00:20:04
32682    00:15:01
32683    00:10:01
32684    00:05:02
32685    00:00:02
Name: Time, Length: 32686, dtype: object

In [90]:
df['Hour']=df['Time'].apply(lambda x:x[0:2])
df['Minute']=df['Time'].apply(lambda x:x[3:5])
df['Second']=df['Time'].apply(lambda x:x[-2:])

In [91]:
df=df.drop('Time',axis=1)

In [92]:
df

,UNIXTime,Data,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet,Day,Month,Year,Hour,Minute,Second
0,1475229326,9/29/2016 12:00:00 AM,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00,9,29,2016,23,55,26
1,1475229023,9/29/2016 12:00:00 AM,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00,9,29,2016,23,50,23
2,1475228726,9/29/2016 12:00:00 AM,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00,9,29,2016,23,45,26
3,1475228421,9/29/2016 12:00:00 AM,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00,9,29,2016,23,40,21
4,1475228124,9/29/2016 12:00:00 AM,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00,9,29,2016,23,35,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,12/1/2016 12:00:00 AM,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00,12,1,2016,00,20,04
32682,1480587301,12/1/2016 12:00:00 AM,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00,12,1,2016,00,15,01
32683,1480587001,12/1/2016 12:00:00 AM,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00,12,1,2016,00,10,01
32684,1480586702,12/1/2016 12:00:00 AM,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00,12,1,2016,00,05,02


In [93]:
df['SunRiseHour']=df['TimeSunRise'].apply(lambda x:x[0:2])
df['SunRiseMinute']=df['TimeSunRise'].apply(lambda x:x[3:5])
df['SunRiseSecond']=df['TimeSunRise'].apply(lambda x:x[-2:])
df['SunSetHour']=df['TimeSunSet'].apply(lambda x:x[0:2])
df['SunSetMinute']=df['TimeSunSet'].apply(lambda x:x[3:5])
df['SunSetSecond']=df['TimeSunSet'].apply(lambda x:x[-2:])
df=df.drop(['TimeSunRise','TimeSunSet','Data'],axis=1)

In [94]:
for column in [column for column in df.columns if df[column].dtypes=='object']:
    df[column]=df[column].astype(np.float)

/tmp/ipykernel_32/3254756707.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df[column]=df[column].astype(np.float)


In [95]:
y=df['Radiation']
x=df.drop('Radiation',axis=1)

x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,random_state=200)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(22880, 18)
(9806, 18)
(22880,)
(9806,)


In [96]:
#scaling
scaler=StandardScaler()
scaler.fit(x_train)
x_train=pd.DataFrame(scaler.transform(x_train),columns=x_train.columns)
x_test=pd.DataFrame(scaler.transform(x_test),columns=x_test.columns)

In [97]:
y_test

17486    288.44
7145       2.79
14641    118.05
26956    853.17
29118      1.24
          ...  
16252      1.18
20748      1.19
14584      1.23
19408      1.22
16470      1.19
Name: Radiation, Length: 9806, dtype: float64

In [98]:
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=200)

In [100]:
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(18304, 18)
(4576, 18)
(18304,)
(4576,)
(9806, 18)
(9806,)


In [101]:
d_train=xgb.DMatrix(x_train,label=y_train)
d_val=xgb.DMatrix(x_val,label=y_val)
d_test=xgb.DMatrix(x_test,label=y_test)